# Reconnaissance de caractères et de symboles

In [131]:
import numpy as np
import cv2
import os
from pyzbar import pyzbar

## Fonctions de détection dans une image

In [174]:
def decode_image(img):
    codes = pyzbar.decode(img)
    return codes

In [ ]:
def update_image(img):
    (x, y, w, h) = code.rect
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                
                
                codeData = code.data.decode("utf-8")
                codeType = code.type

                text = "{} ({})".format(codeData, codeType)
                cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                print("[INFO] Trouvé : {} code : {}".format(codeType, codeData))

## Tests des fonctions

In [175]:
EMPTY_IMAGES_PATH = './data/images/empty'
BARCODE_IMAGES_PATH = './data/images/barcode'
QR_IMAGES_PATH = './data/images/qrcode'

OUTPUT_PATH = './data/outputs/'

In [180]:
def test_code_detection():
    # boucle sur les dossiers d'images
    for path in [EMPTY_IMAGES_PATH, BARCODE_IMAGES_PATH, QR_IMAGES_PATH]:
        # boucle sur les images du dossier
        for img_path in os.listdir(path):
            full_path = path +'/' + img_path
            print(full_path)
            
            img = cv2.imread(full_path)
        
            codes = decode_image(img)
            
            
            
            # boucle sur les codes trouvés
            for code in codes:
                (x, y, w, h) = code.rect
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                
                
                codeData = code.data.decode("utf-8")
                codeType = code.type

                text = "{} ({})".format(codeData, codeType)
                cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                print("[INFO] Trouvé : {} code : {}".format(codeType, codeData))


            cv2.imwrite(OUTPUT_PATH + '/' + img_path, img) 
            
            print("\n")

In [181]:
test_code_detection()

./data/images/empty/img_001.jpg


./data/images/barcode/img_001.png
[INFO] Trouvé : EAN13 code : 0073935942400


./data/images/qrcode/Nestle-and-Google-launch-KitKat-QR-code.jpg




## Fonctions de détection dans une vidéo